In [0]:
%run "/Workspace/Users/sundarasandeepteja@gmail.com/E-Commerce Analytics Medallion Architecture with GenAI/config/project_config"

In [0]:
# Databricks notebook source
# ======================================
# GOLD LAYER: DIM_CUSTOMERS (CUSTOMER DIMENSION)
# ======================================

# MAGIC %run ../config/project_config

from pyspark.sql import functions as F
from pyspark.sql.window import Window

print("🥇 GOLD LAYER: Creating dim_customers")
print("=" * 60)

# ======================================
# LOAD SILVER DATA
# ======================================
print("\n📥 Loading Silver customers with CLV...")

# Use CLV-enhanced customer data
silver_customers = spark.table(f"{SILVER_DB}.customers_with_clv")
print(f"  Silver records: {silver_customers.count():,}")

# ======================================
# CREATE SURROGATE KEY
# ======================================
print("\n🔑 Creating surrogate keys...")

window = Window.orderBy("customer_id")

dim_customers = (
    silver_customers
    .withColumn(
        "customer_key",
        F.row_number().over(Window.orderBy("customer_id"))
    )
    .withColumn(
        "customer_tenure_days",
        F.datediff(F.current_date(), F.col("registration_date"))
    )
    .withColumn(
        "customer_rank_by_revenue",
        F.dense_rank().over(Window.orderBy(F.desc("total_revenue")))
    )
    .withColumn(
        "customer_rank_by_orders",
        F.dense_rank().over(Window.orderBy(F.desc("total_orders")))
    )
    .withColumn(
        "customer_rank_by_clv",
        F.dense_rank().over(Window.orderBy(F.desc("clv_3_year")))
    )
    .withColumn(
        "is_high_value",
        F.when(F.col("clv_segment").isin("Diamond", "Gold"), True).otherwise(False)
    )
    .withColumn(
        "tenure_category",
        F.when(F.col("customer_tenure_days") <= 180, "New")
         .when(F.col("customer_tenure_days") <= 730, "Established")
         .otherwise("Loyal")
    )
    .withColumn(
        "is_at_risk",
        F.when(F.col("churn_risk") == "High", True).otherwise(False)
    )
    .withColumn(
        "days_since_last_order",
        F.datediff(F.current_date(), F.col("last_order"))
    )
    .withColumn(
        "orders_per_month",
        F.when(
            F.col("customer_tenure_days") > 0,
            F.round(F.col("total_orders") / (F.col("customer_tenure_days") / 30), 2)
        ).otherwise(0)
    )
    .withColumn(
        "revenue_per_order",
        F.when(
            F.col("total_orders") > 0,
            F.round(F.col("total_revenue") / F.col("total_orders"), 2)
        ).otherwise(0)
    )
    .withColumn("_loaded_at", F.current_timestamp())
)

print("  ✅ Surrogate keys generated")

# ======================================
# ADD DIMENSION ATTRIBUTES
# ======================================
print("\n➕ Adding dimension attributes...")

dim_customers = dim_customers \
    .withColumn("customer_rank_by_revenue",
        F.dense_rank().over(Window.orderBy(F.desc("total_revenue")))
    ) \
    .withColumn("customer_rank_by_orders",
        F.dense_rank().over(Window.orderBy(F.desc("total_orders")))
    ) \
    .withColumn("customer_rank_by_clv",
        F.dense_rank().over(Window.orderBy(F.desc("clv_3_year")))
    ) \
    .withColumn("is_high_value",
        F.when(F.col("clv_segment").isin("Diamond", "Gold"), True).otherwise(False)
    ) \
    .withColumn("tenure_category",
        F.when(F.col("customer_tenure_days") <= 180, "New")
        .when(F.col("customer_tenure_days") <= 730, "Established")
        .otherwise("Loyal")
    ) \
    .withColumn("is_at_risk",
        F.when(F.col("churn_risk") == "High", True).otherwise(False)
    ) \
    .withColumn("customer_tenure_days",
        F.datediff(F.current_date(), F.col("registration_date"))
    ) \
    .withColumn("days_since_last_order",
        F.datediff(F.current_date(), F.col("last_order"))
    ) \
    .withColumn("orders_per_month",
        F.when(F.col("customer_tenure_days") > 0,
            F.round(F.col("total_orders") / (F.col("customer_tenure_days") / 30), 2)
        ).otherwise(0)
    ) \
    .withColumn("revenue_per_order",
        F.when(F.col("total_orders") > 0,
            F.round(F.col("total_revenue") / F.col("total_orders"), 2)
        ).otherwise(0)
    ) \
    .withColumn(
    "unique_products_purchased",
        F.col("unique_products_purchased") if       "unique_products_purchased" in dim_customers.columns else F.lit(None)
    ) \
    .withColumn("_loaded_at", F.current_timestamp())

# ======================================
# SELECT FINAL COLUMNS
# ======================================
print("\n📋 Selecting final columns...")

final_columns = [
    # Keys
    "customer_key",
    "customer_id",
    
    # Demographics (masked PII)
    "name_masked",
    "email_domain",
    "region",
    "state",
    "age_group",
    
    # Segmentation
    "segment",
    "clv_segment",
    "tenure_category",
    "churn_risk",
    
    # Metrics
    "total_revenue",
    "total_orders",
    "avg_order_value",
    "clv_3_year",
    "revenue_per_order",
    "orders_per_month",
    
    # Product metrics
    "unique_products_purchased",
    
    # Time metrics
    "registration_date",
    "first_order",
    "last_order",
    "customer_tenure_days",
    "days_since_last_order",
    
    # Flags
    "is_active",
    "is_high_value",
    "is_at_risk",
    
    # Rankings
    "customer_rank_by_revenue",
    "customer_rank_by_orders",
    "customer_rank_by_clv",
    
    # Metadata
    "_loaded_at"
]

# Ensure churn_risk is present in silver_customers
if "churn_risk" not in silver_customers.columns:
    raise ValueError("churn_risk column is missing from silver_customers. Please check your source data.")

dim_customers_final = dim_customers.select(final_columns)

# ======================================
# WRITE TO GOLD
# ======================================
print("\n💾 Writing to Gold layer...")

dim_customers_final.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(GOLD_DIM_CUSTOMERS_TABLE)

print(f"  ✅ Created: {GOLD_DIM_CUSTOMERS_TABLE}")

# Optimize
spark.sql(f"OPTIMIZE {GOLD_DIM_CUSTOMERS_TABLE} ZORDER BY (clv_segment, region)")
print("  ✅ Optimized with ZORDER")

# ======================================
# VERIFY AND SUMMARIZE
# ======================================
print("\n📊 Summary Statistics:")

final_count = spark.table(GOLD_DIM_CUSTOMERS_TABLE).count()
print(f"  Total customers: {final_count:,}")

# CLV segment distribution
print("\n📊 Customers by CLV Segment:")
spark.table(GOLD_DIM_CUSTOMERS_TABLE) \
    .groupBy("clv_segment") \
    .agg(
        F.count("*").alias("customers"),
        F.round(F.avg("clv_3_year"), 2).alias("avg_clv"),
        F.round(F.sum("total_revenue"), 2).alias("total_revenue")
    ) \
    .orderBy(F.desc("avg_clv")) \
    .show()

# Churn risk distribution
print("📊 Customers by Churn Risk:")
spark.table(GOLD_DIM_CUSTOMERS_TABLE) \
    .groupBy("churn_risk") \
    .agg(
        F.count("*").alias("customers"),
        F.round(F.avg("total_revenue"), 2).alias("avg_revenue")
    ) \
    .orderBy("churn_risk") \
    .show()

# Region distribution
print("📊 Customers by Region:")
spark.table(GOLD_DIM_CUSTOMERS_TABLE) \
    .groupBy("region") \
    .agg(
        F.count("*").alias("customers"),
        F.round(F.sum("total_revenue"), 2).alias("total_revenue")
    ) \
    .orderBy(F.desc("customers")) \
    .show()

print("\n🥇 DIM_CUSTOMERS COMPLETE!")